In [1]:
from JEFF_Lib import (
    cov_skip_miss, corr_skip_miss, cov_pairwise, corr_pairwise, ewCovar, ewCorr,
    cov_with_different_ew_var_corr, near_psd, _getAplus, _getPS, _getPu, wgtNorm,
    higham_psd, chol_psd, simulate_normal, simulate_pca, FittedModel, fit_normal,
    fit_general_t, general_t_ll, fit_regression_t, return_calculate, VaR_cal, simple_VaR,
    simple_VaR_sim, simple_ES, simple_ES_sim, VaR_ES, Historical_VaR_ES
)

import pandas as pd
import numpy as np
from scipy.stats import norm

# Problem 2

In [2]:
# Load the data from CSV
data = pd.read_csv("problem1.csv")

In [3]:
# Using a normal distribution with an exponentially weighted variance (lambda=0.97)
var_ew_n = simple_VaR(data,"EW_Normal",0.05,0.97)
es_ew_n = simple_ES(data,"EW_Normal",0.05,0.97)
print(var_ew_n, es_ew_n)

[0.09116934 0.09028951] [0.11410652 0.11322669]


In [4]:
# Using a MLE fitted T distribution
var_t = simple_VaR(data, "T", 0.05)
es_t = simple_ES(data, "T", 0.05)
print(var_t, es_t)

[0.07647603 0.0763823 ] [0.1132179  0.11312418]


In [5]:
# Using a Historic Simulation
historical_result = Historical_VaR_ES(data, 100000)
print(historical_result)

[0.0782451  0.07741549 0.11724889 0.11641928]


# Problem 3

In [6]:
# Load the data from CSV
price = pd.read_csv("DailyPrices.csv")
portfolio_data = pd.read_csv("portfolio.csv")

# Data processing
current_price = price.iloc[-1]
returns_data = return_calculate(price, method = "DISCRETE")

In [7]:
def copula_simulation(distribution, portfolio):
    
    nms = portfolio_data[portfolio_data["Portfolio"].isin(["A","B","C"]) if portfolio == "Total" else portfolio_data["Portfolio"] == portfolio]["Stock"].values
    
    # remove date column
    returns = returns_data[nms]

    # all stock names
    stocks = [nm for nm in nms]
    
    # setup how much we hold
    Portfolio = portfolio_data[portfolio_data["Portfolio"].isin(["A","B","C"]) if portfolio == "Total" else portfolio_data["Portfolio"] == portfolio][["Stock","Holding"]]
    
    # remove the mean from all returns
    returns = returns - np.mean(returns, axis = 0)

    # Model fit
    fittedModels = {}
    if distribution == "T":
        for stock in stocks:
            fittedModels[stock] = fit_general_t(returns[stock])[1]
    elif distribution == "Normal":
        for stock in stocks:
            fittedModels[stock] = fit_normal(returns[stock])
    elif portfolio == "Total" and distribution == "Mixed":
        for stock in stocks:
            if portfolio_data[portfolio_data["Stock"] == stock]["Portfolio"].values in (["A","B"]):
                fittedModels[stock] = fit_general_t(returns[stock])[1]
            else:
                fittedModels[stock] = fit_normal(returns[stock])
            
    # Construct the copula
    U = pd.DataFrame({nm: fittedModels[nm].u for nm in nms})
    R = U.corr(method='spearman')
    N = 5000
    simU = pd.DataFrame(norm.cdf(simulate_pca(R, N)), columns=nms)

    # simulation
    simulatedReturns = pd.DataFrame()
    for stock in stocks:
        simulatedReturns[stock] = fittedModels[stock].eval(simU[stock])
    
    # Portfolio Valuation
    iteration = np.arange(1, N + 1)
    values = pd.DataFrame([(stock, Portfolio[Portfolio["Stock"] == stock]["Holding"].values[0], iter) for stock in Portfolio["Stock"] for iter in iteration], columns=['stock', 'holding', 'iteration'])
    values['currentValue'] = values.apply(lambda row: current_price[row['stock']] * row['holding'], axis=1)
    values['simulatedValue'] = values.apply(lambda row: row['currentValue'] * (1.0 + simulatedReturns.loc[row['iteration'] - 1, row['stock']]), axis=1)
    values['pnl'] = values['simulatedValue'] - values['currentValue']
    
    # Risk Metrics
    def calculate_risk_metrics(group):
        return pd.Series({
            'VaR95': VaR_ES(group['pnl'], alpha=0.05)[0],
            'ES95': VaR_ES(group['pnl'], alpha=0.05)[1],
            'VaR99': VaR_ES(group['pnl'], alpha=0.01)[0],
            'ES99': VaR_ES(group['pnl'], alpha=0.01)[1],
            'Standard_Dev': group['pnl'].std(),
            'min': group['pnl'].min(),
            'max': group['pnl'].max(),
            'mean': group['pnl'].mean()
        })

    stockRisk = values.groupby('stock').apply(calculate_risk_metrics).reset_index()
    total_pnl_per_iteration = values.groupby('iteration')['pnl'].sum().reset_index(name='pnl')
    totalRisk = calculate_risk_metrics(total_pnl_per_iteration).to_frame().T
    totalRisk['stock'] = 'Total'
    riskOut = pd.concat([stockRisk, totalRisk], ignore_index=True)
    return riskOut

In [8]:
A = copula_simulation("T", "A")
B = copula_simulation("T", "B")
C = copula_simulation("Normal", "C")
Total = copula_simulation("Mixed", "Total")

/var/folders/f8/gztmghwd1w7chkp8k88bn07m0000gn/T/ipykernel_66835/3836154581.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stockRisk = values.groupby('stock').apply(calculate_risk_metrics).reset_index()
/var/folders/f8/gztmghwd1w7chkp8k88bn07m0000gn/T/ipykernel_66835/3836154581.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stockRisk = values.groupby('stock').apply(calculate_risk_metrics).reset_

In [9]:
A

,stock,VaR95,ES95,VaR99,ES99,Standard_Dev,min,max,mean
0,AAPL,287.750907,410.826313,490.658048,600.040073,182.607682,-888.689123,1060.837840,-0.265619
1,ACN,202.799840,309.978834,363.445979,531.918509,135.981620,-1680.192229,1335.514934,2.143854
2,ADI,417.417857,562.540164,636.986959,798.632700,253.127347,-1323.981394,1258.513269,-8.705410
3,ADP,179.502909,283.968149,332.380077,479.463701,127.084030,-1117.872733,1007.513370,7.572097
4,AMAT,442.371975,646.355951,758.561942,1023.848424,289.445591,-2746.551390,2230.983683,2.192921
5,AMD,519.147047,789.651612,914.039193,1264.779784,339.417064,-3104.517676,3035.642915,-6.049496
6,AVGO,111.086676,166.006497,191.268111,279.839630,73.244419,-487.834336,572.566009,-2.785267
7,BA,236.564990,364.591292,426.910918,599.612324,157.769772,-1115.022052,1212.371956,0.685768
8,BAC,204.573926,294.830859,327.961463,466.306544,131.385107,-1983.195630,881.613639,-5.184421
9,CMCSA,163.980567,240.000245,271.934377,377.836971,108.170667,-750.941924,841.243222,0.178587


In [10]:
B

,stock,VaR95,ES95,VaR99,ES99,Standard_Dev,min,max,mean
0,ADBE,342.573999,518.669123,602.133291,888.068432,226.700854,-1889.058621,2670.493022,2.639787
1,AMGN,321.535687,492.410668,562.146519,821.358223,210.724468,-2229.119631,3319.433076,-8.504852
2,AMT,242.699490,369.050750,416.795060,623.794181,154.561726,-1456.018309,1638.923491,-11.404100
3,AXP,342.889017,517.327368,609.976723,832.683792,220.648024,-1766.886467,1828.217678,1.816904
4,BKNG,439.603668,630.807806,750.541885,989.990968,305.424336,-1696.431270,2260.653943,26.064321
5,CAT,383.828185,553.583689,680.129992,856.102734,247.677832,-1846.444985,1650.360678,5.950760
6,COP,296.896370,407.632903,462.839549,588.211385,179.092079,-1058.286076,991.542611,-2.136125
7,COST,313.829051,488.715677,561.330434,804.182724,212.614681,-2898.059068,2095.701491,5.628533
8,DIS,143.708261,225.281235,273.993091,378.988643,97.695848,-1157.140189,1493.205093,-0.385232
9,GILD,248.720283,372.627899,439.025833,621.819459,161.561131,-1832.001119,1008.382742,-0.476602


In [11]:
C

,stock,VaR95,ES95,VaR99,ES99,Standard_Dev,min,max,mean
0,ABBV,277.609117,347.198041,385.657240,445.495846,172.337014,-614.287556,612.186693,-1.590039
1,ABT,178.129578,223.696367,253.760238,288.554826,105.666641,-471.326388,353.382723,-2.961427
2,AMZN,20.810426,25.944474,29.020370,32.727163,12.694010,-50.411039,46.003566,-0.079377
3,BLK,216.931085,272.535812,305.716976,351.037446,131.533173,-518.655251,509.099465,-1.496861
4,BMY,214.259080,266.555930,299.035670,344.747340,129.108670,-491.138717,562.025882,0.055044
5,BRK-B,199.209690,246.717111,273.278471,313.734853,119.688381,-464.501074,419.040039,-0.542940
6,C,241.383508,298.030871,334.897302,378.591599,141.708857,-545.117038,460.245384,-4.318681
7,CRM,392.103683,487.850215,545.214361,634.103967,238.862925,-828.503012,862.602015,-0.649816
8,CSCO,174.149726,220.034254,248.885889,280.542479,104.884268,-391.389557,444.929785,-0.548314
9,CVX,244.154813,311.289953,348.042427,403.155023,148.752126,-538.601570,503.810995,-0.528317


In [12]:
Total

,stock,VaR95,ES95,VaR99,ES99,Standard_Dev,min,max,mean
0,AAPL,276.084209,418.249449,474.433067,694.824321,186.744900,-1412.090104,1557.017741,5.732199
1,ABBV,276.094627,352.337426,400.187476,464.010014,173.209435,-567.835862,607.538036,4.538200
2,ABT,171.503597,217.276174,242.060611,282.283557,105.491630,-356.286236,418.762935,1.942929
3,ACN,200.331308,302.332954,346.945311,495.709960,138.463069,-2588.681229,1284.936311,6.966086
4,ADBE,333.486487,522.878652,621.732531,911.312061,225.267473,-2126.830225,1435.854100,7.553452
...,...,...,...,...,...,...,...,...,...
95,WFC,230.613114,340.949587,405.048561,531.671591,152.637683,-1710.986890,1377.807054,3.269232
96,WMT,83.133058,132.277014,157.276608,245.121059,57.319478,-942.575388,441.380495,-1.287363
97,XOM,343.083653,439.751994,509.468239,583.098268,211.233700,-785.530655,839.289137,4.340402
98,ZTS,219.734798,343.325402,400.250628,585.162339,157.560373,-2424.068537,1301.916782,2.596463
